In [ ]:
#!pip install yargy

In [35]:
import pandas as pd
import numpy as np

In [36]:
data = pd.read_csv('/content/drive/MyDrive/Bewise.ai/test_data.csv')

In [37]:
data.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


In [38]:
print('Count speeching =', len(data['dlg_id'].unique()))

Count speeching = 6


In [39]:
# анализ данных
# количество реплик
manager = {}
corpus = []


for i, group in data.groupby(by=['dlg_id', 'role']):
  if i[1] == 'manager':
    manager[i[0]] = list(group['text'])
  print(f'В диалоге {i}, количество реплик = {len(group)}')

for i in range(len(manager)):
  text = ''
  for j in range(len(manager[i])):
    text += manager[i][j] + '. '
  corpus.append(text)


В диалоге (0, 'client'), количество реплик = 93
В диалоге (0, 'manager'), количество реплик = 16
В диалоге (1, 'client'), количество реплик = 26
В диалоге (1, 'manager'), количество реплик = 29
В диалоге (2, 'client'), количество реплик = 45
В диалоге (2, 'manager'), количество реплик = 40
В диалоге (3, 'client'), количество реплик = 31
В диалоге (3, 'manager'), количество реплик = 22
В диалоге (4, 'client'), количество реплик = 21
В диалоге (4, 'manager'), количество реплик = 14
В диалоге (5, 'client'), количество реплик = 63
В диалоге (5, 'manager'), количество реплик = 80


In [40]:
len(corpus)

6

## Извлечение сущностей

### Yargy

In [41]:
# Yargy

In [44]:
# из дата фрейма возьмем только реплики менеджеров
# т.к реплики приветствия и прощания менеджеров стандартизированы правилами компании
# Приветствие и имя явно находятся в начале диалога, поэтому при работе функции
# можно парсить только первые 3 строчки.
# Аналогично с окончанием разговора. 

manager_yargy = {}

for i, group in data.groupby(by=['dlg_id', 'role']):
  if i[1] == 'manager':
    manager_yargy[i[0]] = list(group['text'])

{0: ['Алло здравствуйте', 'Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается', 'Угу ну возможно вы рассмотрите и другие варианты видите это хорошая практика сравнивать', 'Угу а на что вы обращаете внимание при выборе', 'Что для вас приоритет', 'Ну у вас срок заканчивается поэтому мы набрали + ко всему', 'А так нет не только поэтому просто я обратила внимание и вы пользуетесь виджетами мы там с вами сотрудничали', 'А если вы 19 являетесь то лучше то идти бесплатно', 'Ага хорошо', 'Индивидуальным поэтому не все то есть сотрудничают с кем то сейчас да кому то например не с кем сейчас справляться кто то сравнивал', 'Да мы услышали вас спасибо за рекомендации', 'Вот понимаю да но дело в том что в основном да за месяц', 'Необходимо обзванивать', 'Хорошо я данную информацию тогда зафиксирую передам будем иметь ввиду', 'И вам спасибо большое за обратную связь', 'Всего хорошего до свидания'], 1: ['Алло здравствуйте',

In [46]:
from yargy.tokenizer import MorphTokenizer
from yargy import (
    Parser,
    or_, rule
)
from yargy.pipelines import morph_pipeline
from yargy.predicates import (
    eq, gram
)

### RULES

In [47]:
# Name

NAME = or_(
    rule(
        morph_pipeline(['звать']),
        gram('Name')
        ),

    rule(
        gram('Name'),
        morph_pipeline(['звать'])
        )
      )

# Company

COMPANY = or_(
    rule(
        morph_pipeline(['компания']),
        gram('NOUN'),
        gram('NOUN').optional()
    ),
    rule(
        morph_pipeline(['компания']),
        gram('NOUN'),
        eq('бизнес').optional()
    ),

    rule(
        morph_pipeline(['компания']),
        eq('диджитал').optional(),
        gram('NOUN')
        )
    )

# Приветствие - прощание

HELLO = morph_pipeline([
    'Здравствуй',
    'Здравствуйте',
    'Добрый день',
    'Доброе утро',
    'Привет',
    'Добрый вечер',
    'Добрый'
])

GOOD_BYE = morph_pipeline([
    'До свидания',
    'Приятного вечера',
    'Всего доброго',
    'Хорошего дня',
    'Хорошего вечера'
])

### Extractor

In [57]:
def extractor(dialog_series, rules):
  
  parser = Parser(rules)

  for replica in dialog_series:
      matches = list(parser.findall(replica))
      if matches and rules not in (NAME, COMPANY):
        return replica
      elif matches:
        for match in matches:
          toks = ' '.join([_.value for _ in match.tokens])
          return [toks,replica]

## Итоговый датафрейм

In [59]:
# Таблица id, приветствие, представление, Имя, Компания, Прощание

cosolidation_table = pd.DataFrame(index=manager_yargy)

In [62]:
for i in range(len(manager_yargy)):

  # hello
  if extractor(manager_yargy[i], HELLO):
    cosolidation_table.loc[i, 'hello'] = extractor(manager_yargy[i], HELLO)

  # extract name + replica
  if extractor(manager_yargy[i], NAME):
    cosolidation_table.loc[i, 'replica_with_name'] = extractor(manager_yargy[i], NAME)[1]
    cosolidation_table.loc[i, 'name'] = extractor(manager_yargy[i], NAME)[0]

  # extract name (есть возможность вывести так же реплику)
  if extractor(manager_yargy[i], COMPANY):
    cosolidation_table.loc[i, 'company'] = extractor(manager_yargy[i], COMPANY)[0]

  # bye
  if extractor(manager_yargy[i], GOOD_BYE):
    cosolidation_table.loc[i, 'bye'] = extractor(manager_yargy[i], GOOD_BYE)

0
1
2
3
4
5


In [63]:
cosolidation_table

,hello,replica_with_name,name,company,bye
0,Алло здравствуйте,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелина,компания диджитал бизнес,Всего хорошего до свидания
1,Алло здравствуйте,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелина,компания диджитал бизнес,До свидания
2,Алло здравствуйте,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелина,компания диджитал бизнес,NaN
3,Алло дмитрий добрый день,Добрый меня максим зовут компания китобизнес у...,максим зовут,компания китобизнес,Угу все хорошо да понедельника тогда всего доб...
4,NaN,NaN,NaN,NaN,Во вторник все ну с вами да тогда до вторника ...
5,NaN,NaN,NaN,NaN,Ну до свидания хорошего вечера
